In [1]:
# importando bibliotecas
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, learning_curve
#usada para o pré-processamento de daddos, minmax e standard para dimensionar e padronizar e o onehot para codificar variáveis em formato numérico
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
#classificador redes neurais com MLP
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, f1_score
from sklearn import metrics
import numpy as np

In [2]:
dt = pd.read_csv('/home/geraldo/Documentos/python_projects/iot-ia/database_iot_pf.csv', low_memory=False)

In [3]:
dt.head()

,lux,humidity,temperature,lighting_switch,climate_switch,door_actuator,lux_status,humidity_status,temperature_status,environment_temperature,environment_lux,label
0,108.385419,50.247112,47.334662,1,1,1,1,1,0,19,400,3
1,86.984219,74.412700,67.968519,1,1,1,1,1,1,24,300,3
2,227.617681,62.304369,71.416293,1,1,1,1,1,1,21,450,3
3,140.915569,58.082378,55.103730,1,1,1,1,1,0,16,250,3
4,233.051872,71.323435,58.827894,1,0,1,1,1,1,25,300,3


In [4]:
dt['label'].value_counts()

label
3    287386
2    271456
1    267847
0    254634
Name: count, dtype: int64

In [6]:
# normal = dt.loc[dt['label'] == 0]
# arcond = dt.loc[dt['label'] == 1]
# ilu = dt.loc[dt['label'] == 2]
# iluar = dt.loc[dt['label'] == 3]

In [7]:
# Selecionando aleatoriamente aproximadamente 51000 amostras de cada classe para balancear os dados de treinamento

# X_train_ilu, X_tst_ilu, y_train_ilu, y_tst_ilu = train_test_split(ilu.drop(['label'],axis=1), ilu['label'], test_size=0.13, shuffle=True)
# X_train_arcond, X_tst_arcond, y_train_arcond, y_tst_arcond = train_test_split(arcond.drop(['label'],axis=1), arcond['label'], test_size=0.29, shuffle=True)
# X_train_normal, X_tst_normal, y_train_normal, y_tst_normal = train_test_split(normal.drop(['label'],axis=1), normal['label'], test_size=0.038, shuffle=True)

In [8]:
# Transformando array em series pandas novamente
# normal_pd_X = pd.DataFrame(X_tst_normal)
# normal_pd_y = pd.DataFrame(y_tst_normal)
# ilu_pd_X = pd.DataFrame(X_tst_ilu)
# ilu_pd_y = pd.DataFrame(y_tst_ilu)
# arcond_pd_X = pd.DataFrame(X_tst_arcond)
# arcond_pd_y = pd.DataFrame(y_tst_arcond)

# normal_pd_X['label'] = normal_pd_y
# ilu_pd_X['label'] = ilu_pd_y
# arcond_pd_X['label'] = arcond_pd_y

# # concatenando as amostras das 4 classes de saída
# data = pd.concat([iluar, normal_pd_X, ilu_pd_X, arcond_pd_X])
# data['label'].value_counts()

In [9]:
# data.info()

In [10]:
# Número de features
inputs = len(dt.columns)-1
print(inputs)

11


In [11]:
#função para plotar a curva de aprendizagem
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    
    plt.legend(loc="best")
    return plt


In [12]:
#hiperparâmetros
svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

forest_params = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [13]:
models = {
#     'MLP': MLPClassifier(hidden_layer_sizes=(inputs,32,16,4),
#                         max_iter = 100,activation = 'logistic',
#                         solver = 'adam',learning_rate = 'adaptive',
#                         learning_rate_init = 0.007, early_stopping = True,
#                         verbose=True),
    'SVM': SVC(random_state=42),
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'Random Forest': RandomForestClassifier(random_state=42)
}

In [14]:
# X = pd.concat([X_train, X_test])
# y = pd.concat([y_train, y_test])

X_train, X_test, y_train, y_test = train_test_split(dt.drop(['label'],axis=1), dt['label'], test_size=0.13, shuffle=True)


In [ ]:
for name, model in models.items():
    plot_learning_curve(model, f'Learning Curve ({name})', X_train, y_train, cv=3, train_sizes=np.linspace(0.1, 1.0, 5))
    plt.show()

In [ ]:
# Modelo de MLP para classificação criado
mlp_clf = MLPClassifier(hidden_layer_sizes=(inputs,32,16,4),
                        max_iter = 100,activation = 'logistic',
                        solver = 'adam',learning_rate = 'adaptive',
                        learning_rate_init = 0.007, early_stopping = True,
                        verbose=True)

In [ ]:
# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(data.drop(['label'],axis=1), data['label'], test_size=0.1, shuffle=True)

In [ ]:
full_pipe = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler()),
    ('mlp', mlp_clf)
])

In [ ]:
# Realizando o treinamento
full_pipe.fit(X_train,y_train)

In [ ]:
# Usando preditor para os dados de treino
y_train_pred = full_pipe.predict(X_train)
print(y_train_pred)

In [ ]:
y_train_score = full_pipe.score(X_train, y_train)
print(y_train_score)

In [ ]:
# Usando o preditor nos dados de teste
y_test_pred = full_pipe.predict(X_test)
print(y_test_pred)

In [ ]:
# Observando a acurácia do modelo nos dados de teste
y_test_score = full_pipe.score(X_test, y_test)
print(y_test_score)

In [ ]:
# Curva de perdas do melhor modelo
plt.plot(mlp_clf.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

In [ ]:
print(
    f"Classification report for classifier {mlp_clf}:\n"
    f"{metrics.classification_report(y_test, y_test_pred)}\n"
)

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
disp.figure_.suptitle("Matriz de confusão")
print(f"Matriz de confusão:\n{disp.confusion_matrix}")

plt.show()

In [ ]:
import joblib

joblib.dump(full_pipe, '/home/geraldo/Documentos/python_projects/iot-ia/mlp_smart_enviroument.joblib')

In [ ]:
m = joblib.load('/home/geraldo/Documentos/python_projects/iot-ia/mlp_smart_enviroument.joblib')

In [ ]:
# lux | humidity | temperature | ligthing_switch | climate_switch | door_actuator | lux_status | humidity_status
# temperature_status | enviroument_temperature | enviroument_lux
# testando dados
features = [286, 50, 30, 1, 1, 1, 1, 1, 1, 25, 300]

# convertendo para array
features_array = np.array([features])

# Prevendo a saída
prediction = m.predict(features_array)

# Mostrando o resultado
print(prediction)


In [ ]:
data['label'].value_counts()

In [ ]:
data.shape

In [ ]:
dt.shape